In [1]:
import numpy as np
import pandas as pd
import keras
import os
from pathlib import Path

In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [3]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [4]:
data_path = "fra.txt"

In [5]:
input_texts = []
target_texts = []
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = target_text
    input_texts.append(input_text)
    target_texts.append(target_text)

In [6]:
exclude = set(string.punctuation)

In [7]:
remove_digits = str.maketrans('', '', digits)

In [8]:
input_data = []
for i in range(0, len(input_texts)):
    review = input_texts[i]
    review = review.lower()
    review = re.sub("'", '', review)
    review = re.sub(",", ' COMMA', review) 
    review = [ch for ch in review if ch not in exclude]
    review = ''.join(review)
    review = review.translate(remove_digits)
    input_data.append(review)

In [9]:
target_data = []
for i in range(0, len(target_texts)):
    review = target_texts[i]
    review = review.lower()
    review = re.sub("'", '', review)
    review = re.sub(",", ' COMMA', review)
    review = [ch for ch in review if ch not in exclude]
    review = ''.join(review)
    review = review.translate(remove_digits)
    review = 'START_ '+ review + ' _END'
    target_data.append(review)

In [10]:
target_data[0]

'START_ va  _END'

In [11]:
english_words=set()
for eng in input_data:
    for word in eng.split():
        if word not in english_words:
            english_words.add(word)

In [12]:
french_words=set()
for french in target_data:
    for word in french.split():
        if word not in french_words:
            french_words.add(word)

In [13]:
print(len(english_words))
print(len(french_words))

2016
4411


In [14]:
lenght_list_english=[]
for l in input_data:
    lenght_list_english.append(len(l.split(' ')))
np.max(lenght_list_english)

5

In [15]:
lenght_list_french=[]
for l in target_data:
    lenght_list_french.append(len(l.split(' ')))
np.max(lenght_list_french)

12

In [16]:
all_english_words = sorted(list(english_words))
all_french_words = sorted(list(french_words))
english_vocab_size = len(all_english_words)
french_vocab_size = len(all_french_words)
english_max_sent_length = np.max(lenght_list_english)
french_max_sent_length = np.max(lenght_list_french)

In [17]:
english_token_index = dict([(word, i) for i, word in enumerate(all_english_words)])
french_token_index = dict([(word, i) for i, word in enumerate(all_french_words)])

In [18]:
encoder_input_data = np.zeros((num_samples, english_max_sent_length), dtype='float32')
decoder_input_data = np.zeros((num_samples, french_max_sent_length), dtype='float32')
decoder_target_data = np.zeros((num_samples, french_max_sent_length, french_vocab_size), dtype='float32')

In [19]:
for i, (input_text, target_text) in enumerate(zip(input_data, target_data)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = english_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = french_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, french_token_index[word]] = 1.

In [20]:
embedding_size = 256

In [21]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [22]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(english_vocab_size, embedding_size)(encoder_inputs)
encoder = LSTM(embedding_size, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [23]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(french_vocab_size, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(embedding_size, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(french_vocab_size, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size,
          epochs,
          validation_split=0.05)

Epoch 1/10
149/149 [==============================] - 54s 332ms/step - loss: 1.8538 - accuracy: 0.0951 - val_loss: 2.1036 - val_accuracy: 0.0833
Epoch 2/10
149/149 [==============================] - 54s 365ms/step - loss: 1.7175 - accuracy: 0.1064 - val_loss: 2.0257 - val_accuracy: 0.0988
Epoch 3/10
149/149 [==============================] - 59s 395ms/step - loss: 1.6275 - accuracy: 0.1171 - val_loss: 1.9564 - val_accuracy: 0.1332
Epoch 4/10
149/149 [==============================] - 46s 312ms/step - loss: 1.5415 - accuracy: 0.1197 - val_loss: 1.8783 - val_accuracy: 0.1332
Epoch 5/10
149/149 [==============================] - 48s 320ms/step - loss: 1.4659 - accuracy: 0.1241 - val_loss: 1.7716 - val_accuracy: 0.1743
Epoch 6/10
149/149 [==============================] - 47s 312ms/step - loss: 1.4011 - accuracy: 0.1304 - val_loss: 1.7177 - val_accuracy: 0.1778
Epoch 7/10
149/149 [==============================] - 46s 309ms/step - loss: 1.3390 - accuracy: 0.1338 - val_loss: 1.6806 - val_ac

In [25]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         516096    
                                                                 
 lstm (LSTM)                 [(None, 256),             525312    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 1,041,408
Trainable params: 1,041,408
Non-trainable params: 0
_________________________________________________________________


In [27]:
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in english_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in french_token_index.items())

In [28]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = french_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [31]:
for seq_index in [1,20,30,56,199]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_data[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence[:-4])

-
Input sentence: ['go']
Decoded sentence:  
-
Input sentence: ['run']
Decoded sentence:  dégage 
-
Input sentence: ['help']
Decoded sentence:  dégage 
-
Input sentence: ['i see']
Decoded sentence:  je me suis 
-
Input sentence: ['beat it']
Decoded sentence:  dégage 
